In [1]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [2]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [3]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [4]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [5]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [6]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [7]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [8]:
print(decision_steps)
print(list(decision_steps))

[0]


In [9]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [10]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [11]:
env.set_actions(behavior_name, action)

In [12]:
for episode in range(30):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        action = spec.action_spec.random_action(len(decision_steps))
        #action = spec.action_spec.empty_action(len(decision_steps))
        #action.add_continuous(np.array([[-0.5]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

[array([[ 1.5392594, -1.1772   , 19.94068  , -1.       ,  8.354665 ,
         8.040739 , 13.202666 ,  9.867748 ]], dtype=float32)]
[array([[ 1.8703911, -2.1582   , 19.91235  , -1.       ,  8.469465 ,
         7.731675 , 12.514383 , 10.024717 ]], dtype=float32)]
[array([[-0.13137059, -3.1391995 , 19.999569  , 28.195131  ,  8.843477  ,
         7.611805  , 16.009558  ,  8.706542  ]], dtype=float32)]
[array([[-0.37312698,  0.0388871 , 19.801249  , 27.651196  ,  9.150086  ,
         7.150582  , 16.45723   ,  8.31879   ]], dtype=float32)]
[array([[-2.6652367 , -0.07722417, 19.767141  , 18.748491  , -1.        ,
         6.905206  , -1.        ,  7.085281  ]], dtype=float32)]
[array([[-1.9548404 ,  0.04268797, 19.683931  , 19.637997  , -1.        ,
         6.0591984 , 23.707989  ,  6.6530805 ]], dtype=float32)]
[array([[-3.811851  ,  0.03001493, 19.459372  , 13.707817  , 11.78913   ,
         5.6619654 , -1.        ,  5.5408134 ]], dtype=float32)]
[array([[-6.7380958 , -0.05110325, 18.71521

KeyboardInterrupt: 

In [ ]:
env.close()